In [2]:
import pandas as pd
import json
import csv

In [3]:
t1 = ["2018/" + str(i) for i in range(201800000,201802792)]
t2 = ["2019/" + str(i) for i in range(201900000,201903082)]
t3 = ["2020/" + str(i) for i in range(202000000,202003393)]
t4 = ["2021/" + str(i) for i in range(202100000,202103815)]
t5 = ["2022/" + str(i) for i in range(202200000,202204244)]
t6 = ["2023/" + str(i) for i in range(202300000,202302890)]

t = t1 + t2 + t3 + t4 + t5 + t6

In [4]:
len(t)

20216

In [28]:
from queue import Full


i = 0
for dir in t:
    f = open('../data/' + dir, 'r', encoding="utf8")
    data = json.load(f)
    f.close()

    isHasReference = data['abstracts-retrieval-response']['item']['bibrecord']['tail'] != None

    affiliation_json = data['abstracts-retrieval-response']['affiliation']
    authors_json = data['abstracts-retrieval-response']['authors']['author']
    if(isHasReference):
        reference_json = data['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference']
    
    isSingleAffliation = type(affiliation_json) != list
    isSingleAuthors = type(authors_json) != list
    if(isHasReference):
        isSingleReference = type(reference_json) != list

    #main csv file
    title = data['abstracts-retrieval-response']['item']['bibrecord']['head']['citation-title']
    abstract = data['abstracts-retrieval-response']['item']['bibrecord']['head']['abstracts']
    date_data = data['abstracts-retrieval-response']['item']['bibrecord']['head']['source']['publicationdate']
    
    try:
        day = date_data['day']
    except:
        day = ""
    
    try:
        month = date_data['month']
    except:
        month = ""

    try:
        year = date_data['year']
    except:
        year = ""

    authors = []
    if (isSingleAuthors):
        authors = [authors_json['ce:indexed-name']]
    else:
        authors = [x['ce:indexed-name'] for x in authors_json]
    author_keywords = [keyword['$'] for keyword in data['abstracts-retrieval-response']['item'].get('authkeywords', {}).get('author-keyword', [])]
    reference_authors = []
    if (isSingleReference):
        if 'ref-authors' in reference_json.get('ref-info', {}):
            for author in reference_json['ref-info']['ref-authors'].get('author', []):
                reference_authors.append(author.get('ce:indexed-name', ''))
    elif (isHasReference):
        for reference in reference_json:
            if 'ref-authors' in reference.get('ref-info', {}):
                for author in reference['ref-info']['ref-authors'].get('author', []):
                    reference_authors.append(author.get('ce:indexed-name', ''))

    abbrev = set()
    for subject in data['abstracts-retrieval-response']['subject-areas']['subject-area']:
        abbrev.add(subject['@abbrev'])

    abbrev = list(abbrev)

    with open('main_file.csv', 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow([title, abstract, day, month, year, authors, author_keywords, reference_authors, abbrev])

    #author to allications file
    author_affiliation_data = []
    if (isSingleAuthors):
        try:
            if (type(authors_json['affiliation']) == list):
                for aut in authors_json['affiliation']:
                    author_affiliation_data.append([authors_json['ce:indexed-name'], aut['@id']])
            else:
                author_affiliation_data.append([authors_json['ce:indexed-name'], authors_json['affliation']['@id']])
        except:
            c = 1
    else:
        for group in authors_json:
            try:
                if (type(group['affiliation']) == list):
                    for aut in group['affiliation']:
                        author_affiliation_data.append([group['ce:indexed-name'], aut['@id']])
                else:
                    author_affiliation_data.append([group['ce:indexed-name'], group['affiliation']['@id']])
            except:
                c = 1

    with open('author2afflication.csv', 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerows(author_affiliation_data)

    #afflication file
    affiliation_data = []
    if (isSingleAffliation):
        country = affiliation_json['affiliation-country']
        city = affiliation_json['affiliation-city']
        organization = affiliation_json['affilname']
        afid = affiliation_json['@id']
        affiliation_data.append([afid, country, city, organization])
    else:
        for aff in affiliation_json:
            country = aff['affiliation-country']
            city = aff['affiliation-city']
            organization = aff['affilname']
            afid = aff['@id']
            affiliation_data.append([afid, country, city, organization])

    with open('afflication.csv', 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerows(affiliation_data)

    #reference file
    reference_data = []
    try:
        for reference in reference_json:
            ref_info = reference['ref-info']
            if 'ref-website' in ref_info:
                website_data = ref_info['ref-website']
                if isinstance(website_data, list):
                    for website in website_data:
                        attributes = list(website.keys())
                        link = website['$']
                        reference_data.append(attributes + [link])
                else:
                    attributes = list(website_data.keys())
                    link = website_data['ce:e-address']['$']
                    reference_data.append(attributes + [link])
    except:
        print("skip at file" + str(i + 1))

    with open('reference.csv', 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerows(reference_data)

    i += 1
    if (i % 256 == 0):
        print(str(i) + " files read successfully")

2018/201800000
2018/201800001
2018/201800002
2018/201800003
2018/201800004
2018/201800005
2018/201800006
2018/201800007
2018/201800008
2018/201800009
2018/201800010
2018/201800011
2018/201800012
2018/201800013
2018/201800014
2018/201800015
2018/201800016
2018/201800017
2018/201800018
2018/201800019
2018/201800020
2018/201800021
2018/201800022
2018/201800023
2018/201800024
2018/201800025
2018/201800026
2018/201800027
2018/201800028
2018/201800029
skip at file30
2018/201800030
2018/201800031
2018/201800032
2018/201800033
2018/201800034
2018/201800035
2018/201800036
2018/201800037
2018/201800038
2018/201800039
2018/201800040
2018/201800041
2018/201800042
2018/201800043
2018/201800044
2018/201800045
2018/201800046
2018/201800047
2018/201800048
2018/201800049
2018/201800050
2018/201800051
2018/201800052
2018/201800053
2018/201800054
2018/201800055
2018/201800056
2018/201800057
2018/201800058
2018/201800059
2018/201800060
2018/201800061
2018/201800062
2018/201800063
2018/201800064
2018/20180

KeyError: 'day'